# Лабораторная работа №7

## Рекуррентные нейронные сети для анализа текста

Набор данных для предсказания оценок для отзывов, собранных с сайта _imdb.com_, который состоит из 50,000 отзывов в виде текстовых файлов.

Отзывы разделены на положительные (25,000) и отрицательные (25,000).

Данные предварительно токенизированы по принципу «мешка слов», индексы слов можно взять из словаря (_imdb.vocab_).

Обучающая выборка включает в себя 12,500 положительных и 12,500 отрицательных отзывов, контрольная выборка также содержит 12,500 положительных и 12,500 отрицательных отзывов.

Данные можно скачать ~~на сайте _Kaggle_~~: ~~https://www.kaggle.com/iarunava/imdb-movie-reviews-dataset~~ https://ai.stanford.edu/~amaas/data/sentiment/

### Задание 1

Загрузите данные. Преобразуйте текстовые файлы во внутренние структуры данных, которые используют индексы вместо слов.

In [1]:
from google.colab import drive

drive.mount('/content/drive', force_remount = True)

Mounted at /content/drive


In [0]:
BASE_DIR = '/content/drive/My Drive/Colab Files/mo-2'

import sys

sys.path.append(BASE_DIR)

import os

In [0]:
DATA_ARCHIVE_NAME = 'imdb-dataset-of-50k-movie-reviews.zip'

LOCAL_DIR_NAME = 'imdb-sentiments'

In [0]:
from zipfile import ZipFile

with ZipFile(os.path.join(BASE_DIR, DATA_ARCHIVE_NAME), 'r') as zip_:
    zip_.extractall(LOCAL_DIR_NAME)

In [0]:
DATA_FILE_PATH = 'imdb-sentiments/IMDB Dataset.csv'

In [0]:
import pandas as pd

all_df = pd.read_csv(DATA_FILE_PATH)

In [7]:
print(all_df.shape)

(50000, 2)


In [0]:
train_df = all_df.sample(frac = 0.7)
all_df = all_df.drop(train_df.index)

val_df = all_df.sample(frac = 0.5)
test_df = all_df.drop(val_df.index)

In [9]:
print(train_df.shape)
print(val_df.shape)
print(test_df.shape)

(35000, 2)
(7500, 2)
(7500, 2)


In [10]:
import nltk

nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [0]:
MAX_LENGTH = 100

STRING_DTYPE = '<U12'

PADDING_TOKEN = 'PAD'

In [0]:
from nltk import word_tokenize
import numpy as np
import string
import re

def tokenize_string(_string):
    return  [tok_.lower() for tok_ in word_tokenize(_string) if not re.fullmatch('[' + string.punctuation + ']+', tok_)]

def pad(A, length):
    arr = np.empty(length, dtype = STRING_DTYPE)
    arr.fill(PADDING_TOKEN)
    arr[:len(A)] = A
    return arr

def tokenize_row(_sentence):
    return pad(tokenize_string(_sentence)[:MAX_LENGTH], MAX_LENGTH)

def encode_row(_label):
    return 1 if _label == 'positive' else 0

def encode_and_tokenize(_dataframe):

    tttt = _dataframe.apply(lambda row: tokenize_row(row['review']), axis = 1)
    llll = _dataframe.apply(lambda row: encode_row(row['sentiment']), axis = 1)

    data_dict = { 'label': llll, 'tokens': tttt }

    encoded_and_tokenized = pd.DataFrame(data_dict, columns = ['label', 'tokens'])

    return encoded_and_tokenized

In [15]:
encode_and_tokenize(val_df)

,label,tokens
39231,1,"[i, love, this, movie, is, it, on, air, anymor..."
13963,0,"[after, the, lush, inspiring, aerial, shot, in..."
29599,1,"[how, many, of, us, have, read, a, book, or, s..."
3931,1,"[making, a, movie, about, a, comic, is, hard, ..."
20866,1,"[dominion, tank, police, is, without, a, shell..."
...,...,...
16874,1,"[after, clocking, up, five, seasons, on, the, ..."
10555,1,"[i, happened, to, see, this, movie, twice, or,..."
30114,1,"[set, in, 2017, although, one, might, easily, ..."
43089,1,"[the, young, victoria, is, a, elegantly, costu..."


### Задание 2

Реализуйте и обучите двунаправленную рекуррентную сеть (_LSTM_ или _GRU_).

Какого качества классификации удалось достичь?

### Задание 3

Используйте индексы слов и их различное внутреннее представление (_word2vec_, _glove_). Как влияет данное преобразование на качество классификации?

### Задание 4

Поэкспериментируйте со структурой сети (добавьте больше рекуррентных, полносвязных или сверточных слоев). Как это повлияло на качество классификации?

### Задание 5

Используйте предобученную рекуррентную нейронную сеть (например, _DeepMoji_ или что-то подобное).

Какой максимальный результат удалось получить на контрольной выборке?